<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Turku_ERA5_CDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium

# The precise City Center coordinates we chose
lat, lon = 60.5, 22.3

# 1. Initialize the map centered on Turku
# We use 'cartodbpositron' for a clean, professional look that highlights markers
m = folium.Map(location=[lat, lon], zoom_start=12, control_scale=True)

# 2. Add a marker for the exact Grid Center
folium.Marker(
    [lat, lon],
    popup="<b>ERA5-Land Grid Center</b><br>Lat: 60.5, Lon: 22.3",
    tooltip="Turku City Center Point",
    icon=folium.Icon(color='darkred', icon='info-sign')
).add_to(m)

# 3. Add a rectangle to show the ERA5-Land Grid Cell (approx 0.1 x 0.1 degrees)
# This represents the actual area the weather data is calculated for.
folium.Rectangle(
    bounds=[[60.45, 22.25], [60.55, 22.35]],
    color="blue",
    fill=True,
    fill_opacity=0.1,
    popup="Approximate 9km Grid Cell Area"
).add_to(m)

# 4. Add a reference marker for the Market Square (Kauppatori)
# to see how close it is to our data point.
folium.CircleMarker(
    location=[60.4515, 22.2672],
    radius=5,
    color="green",
    fill=True,
    popup="Market Square (City Center)"
).add_to(m)

# Display the map in Colab
m

In [14]:
! pip install "cdsapi>=0.7.7"

In [15]:
!pip install netcdf4

In [16]:
import os

# Updated configuration: removed the UID prefix as per the error instructions
content = """url: https://cds.climate.copernicus.eu/api
key: c025f203-5930-4d9c-acd6-699c46be7fd8"""

with open(os.path.expanduser('~/.cdsapirc'), 'w') as f:
    f.write(content)

print("Configuration updated! Now attempting to update the library...")

# Also update your library to the latest version to match the new API
!pip install --upgrade cdsapi

Configuration updated! Now attempting to update the library...


In [ ]:
import cdsapi
import os
import calendar

# CRITICAL FIX: Set progress=False to prevent widget metadata creation
c = cdsapi.Client(progress=False)

# Range: Full year 2024 + January 2025
tasks = [('2024', str(m).zfill(2)) for m in range(1, 13)] + [('2025', '01')]

print("--- Data Download Started (GitHub-Safe Mode) ---")

for i, (year, month) in enumerate(tasks):
    filename = f'turku_{year}_{month}.nc'

    if os.path.exists(filename):
        print(f"[{i+1}/13] {filename} already exists. Skipping.")
        continue

    # Determine days: Full month for 2024, but only Jan 1st for 2025
    if year == '2025':
        days = ['01']
    else:
        last_day = calendar.monthrange(int(year), int(month))[1]
        days = [str(d).zfill(2) for d in range(1, last_day + 1)]

    print(f"[{i+1}/13] Requesting {month}/{year}...")

    try:
        c.retrieve(
            'reanalysis-era5-land',
            {
                'variable': [
                    '2m_temperature',
                    '10m_u_component_of_wind',
                    '10m_v_component_of_wind',
                    'surface_solar_radiation_downwards',
                ],
                'year': year,
                'month': month,
                'day': days,
                'time': [f"{str(h).zfill(2)}:00" for h in range(24)],
                'area': [60.5, 22.3, 60.5, 22.3],
                'format': 'netcdf',
            },
            filename)
        # We print a success message that will stay in your GitHub logs
        print(f"      ✅ {filename} successfully downloaded and saved.")
    except Exception as e:
        print(f"      ❌ Failed {year}-{month}: {e}")

print("--- All Downloads Complete ---")

2025-12-20 06:26:19,812 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
INFO:ecmwf.datastores.legacy_client:[2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


--- Data Download Started (GitHub-Safe Mode) ---
[1/13] Requesting 01/2024...


2025-12-20 06:26:20,637 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2025-12-20 06:26:20,640

      ✅ turku_2024_01.nc successfully downloaded and saved.
[2/13] Requesting 02/2024...


2025-12-20 06:32:46,005 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
INFO:ecmwf.datastores.legacy_client:[2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-timeseries?tab=overview)
2025-12-20 06:32:46,007

In [ ]:
!pip install netcdf4 h5netcdf

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
import zipfile

# 1. Setup paths
file_list = sorted(glob.glob('turku_*.nc'))
extract_dir = 'extracted_temp'
os.makedirs(extract_dir, exist_ok=True)
all_data = []

print(f"--- Processing {len(file_list)} files ---")

for f in file_list:
    try:
        # ZIP check
        with open(f, 'rb') as test_f:
            is_zip = test_f.read(2) == b'PK'

        if is_zip:
            with zipfile.ZipFile(f, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
            inner_files = glob.glob(os.path.join(extract_dir, "*.nc"))
            if inner_files:
                ds = xr.open_dataset(inner_files[0], engine='netcdf4')
                df = ds.to_dataframe().reset_index()
                all_data.append(df)
                for extra in inner_files: os.remove(extra)
        else:
            ds = xr.open_dataset(f, engine='netcdf4')
            df = ds.to_dataframe().reset_index()
            all_data.append(df)

        print(f"✅ Processed: {f}")
    except Exception as e:
        print(f"❌ Error on {f}: {e}")

if all_data:
    df_combined = pd.concat(all_data).sort_values('valid_time')

    # 2. Convert and Split Date and Hour
    df_combined['valid_time'] = pd.to_datetime(df_combined['valid_time'])

    # Extract Date as YYYY-MM-DD
    df_combined['Date'] = df_combined['valid_time'].dt.strftime('%Y-%m-%d')

    # Extract Hour and format as HH:00 (e.g., 01:00)
    df_combined['Hour'] = df_combined['valid_time'].dt.strftime('%H:00')

    # 3. Unit Conversions
    df_combined['Temperature_C'] = df_combined['t2m'] - 273.15
    df_combined['Wind_Speed_ms'] = np.sqrt(df_combined['u10']**2 + df_combined['v10']**2)
    df_combined['GHI_Wm2'] = df_combined['ssrd'] / 3600

    # 4. Final selection and reorder
    final_cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms', 'GHI_Wm2']
    final_output = df_combined[final_cols]

    # 5. Export
    csv_name = 'Turku_Weather_Hourly_2024_Clean.csv'
    final_output.to_csv(csv_name, index=False)

    print("\n" + "="*40)
    print(f"SUCCESS: Saved to {csv_name}")
    print(f"Total Rows: {len(final_output)}")
    print("="*40)

    # Display preview for GitHub logs
    print("\nData Preview (showing top 5 rows):")
    print(final_output.head())
else:
    print("No data frames found. Make sure the .nc files were downloaded correctly.")